In [6]:

# First we will import the necessary Library

import os
import os
# os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt

# For Evalution we will use these library

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

# For model building we will use these library

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM
import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


# For PLotting we will use these library

import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


# 'CKB', 'DAI', 'NPXS'



In [15]:

# coin = ['AGI', 'AUC', 'BCN', 'BNT', 'BZRX', 'CELR', 'CGLD','CKB', 'DAI', 'NPXS', 'CHAT', 'CHR', 'DMG', 'DOTUP', 'DX', 'ENJ', 'ETP', 'FUEL', 'GBP', 'HSC', 'KIN', 'KMD', 'LEND', 'MBL', 'NEST', 'OCEAN', 'ONL', 'QUN', 'RSR', 'SFG', 'SWAP', 'TFUEL', 'TNT', 'UTK', 'WXT', 'XSR', 'YEE', 'YFII', 'ZEN']
# coin = ['NPXS', 'CHAT', 'CHR', 'DMG', 'DOTUP', 'DX', 'ENJ', 'ETP', 'FUEL', 'GBP', 'HSC', 'KIN', 'KMD', 'LEND', 'MBL', 'NEST', 'OCEAN', 'ONL', 'QUN', 'RSR', 'SFG', 'SWAP', 'TFUEL', 'TNT', 'UTK', 'WXT', 'XSR', 'YEE', 'YFII', 'ZEN']
coin = ["DIA"]
for file_name in coin:
    maindf=pd.read_csv(f"/content/train/{file_name}.csv")

    sd=maindf.iloc[0][0]
    ed=maindf.iloc[-1][0]


    """# Overall Analysis from 2014-2022"""

    maindf['datetime'] = pd.to_datetime(maindf['datetime'], format='%Y-%m-%d')

    y_overall = maindf.loc[(maindf['datetime'] >= sd)
                        & (maindf['datetime'] <= ed)]

    y_overall.drop(y_overall[['close_x','volumeto_x']],axis=1)

    monthvise= y_overall.groupby(y_overall['datetime'].dt.strftime('%B'))[['open_x','close_x']].mean()
    new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
                'September', 'October', 'November', 'December']
    monthvise = monthvise.reindex(new_order, axis=0)
    monthvise

    names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

    # Lets First Take all the Close Price
    closedf = maindf[['datetime','close_x']]
    closedf = closedf[closedf['datetime'] > '2019-10-11']
    close_stock = closedf.copy()

    del closedf['datetime']
    scaler=MinMaxScaler(feature_range=(0,1))
    closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))

    # Recalculate the sizes
    training_size = int(len(closedf) * 0.60)
    test_size = len(closedf) - training_size


    # Split the data
    train_data = closedf[:training_size]
    test_data = closedf[training_size:]


    def create_dataset(dataset, time_step=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_step-1):
            a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])
        return np.array(dataX), np.array(dataY)

    time_step = 15
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)

    # reshape input to be [samples, time steps, features] which is required for LSTM
    X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
    X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)


    """* # Actuall Model Building"""

    model=Sequential()

    model.add(LSTM(10,input_shape=(None,1),activation="relu"))

    model.add(Dense(1))

    model.compile(loss="mean_squared_error",optimizer="adam")

    history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=32,verbose=1)

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(loss))


    ### Lets Do the prediction and check performance metrics
    train_predict=model.predict(X_train)
    test_predict=model.predict(X_test)
    train_predict.shape, test_predict.shape

    """- # Model Evaluation"""

    # Transform back to original form

    train_predict = scaler.inverse_transform(train_predict)
    test_predict = scaler.inverse_transform(test_predict)
    original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
    original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))



    """- # Comparision of original stock close price and predicted close price"""

    close_stock

    look_back=time_step
    trainPredictPlot = np.empty_like(closedf)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
    print("Train predicted data: ", trainPredictPlot.shape)

    # shift test predictions for plotting
    testPredictPlot = np.empty_like(closedf)
    testPredictPlot[:, :] = np.nan
    testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
    print("Test predicted data: ", testPredictPlot.shape)

    names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


    plotdf = pd.DataFrame({'date': close_stock['datetime'],
                        'original_close': close_stock['close_x'],
                        'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                        'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})


    """- # Predicting next 30 days"""

    x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
    temp_input=list(x_input)
    temp_input=temp_input[0].tolist()

    from numpy import array

    lst_output=[]
    n_steps=time_step
    i=0
    pred_days = 30
    while(i<pred_days):

        if(len(temp_input)>time_step):

            x_input=np.array(temp_input[1:])
            #print("{} day input {}".format(i,x_input))
            x_input = x_input.reshape(1,-1)
            x_input = x_input.reshape((1, n_steps, 1))

            yhat = model.predict(x_input, verbose=0)
            #print("{} day output {}".format(i,yhat))
            temp_input.extend(yhat[0].tolist())
            temp_input=temp_input[1:]
            #print(temp_input)

            lst_output.extend(yhat.tolist())
            i=i+1

        else:

            x_input = x_input.reshape((1, n_steps,1))
            yhat = model.predict(x_input, verbose=0)
            temp_input.extend(yhat[0].tolist())

            lst_output.extend(yhat.tolist())
            i=i+1

    print("Output of predicted next days: ", len(lst_output))

    """- # Plotting last 15 days of dataset and next predicted 30 days"""

    last_days=np.arange(1,time_step+1)
    day_pred=np.arange(time_step+1,time_step+pred_days+1)
    print(last_days)
    print(day_pred)

    temp_mat = np.empty((len(last_days)+pred_days+1,1))
    temp_mat[:] = np.nan
    temp_mat = temp_mat.reshape(1,-1).tolist()[0]

    last_original_days_value = temp_mat
    next_predicted_days_value = temp_mat

    last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
    next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

    new_pred_plot = pd.DataFrame({
        'last_original_days_value':last_original_days_value,
        'next_predicted_days_value':next_predicted_days_value
    })

    names = cycle(['Last 15 days close price','Predicted next 30 days close price'])



    """- # Plotting entire Closing Stock Price with next 30 days period of prediction"""

    lstmdf=closedf.tolist()
    lstmdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
    lstmdf=scaler.inverse_transform(lstmdf).reshape(1,-1).tolist()[0]

    names = cycle(['Close price'])

    from datetime import timedelta

    # Function to find continuous date ranges
    def find_continuous_date_ranges(df):
        df = df.sort_values(by='datetime').reset_index(drop=True)
        discontinuity_map = {}

        prev_date = df['datetime'].iloc[0]
        continuous_count = 1
        last_discontinuity = prev_date

        for i in range(1, len(df)):
            current_date = df['datetime'].iloc[i]
            if current_date == prev_date + timedelta(days=1):
                continuous_count += 1
            else:
                discontinuity_map[last_discontinuity] = continuous_count
                last_discontinuity = current_date
                continuous_count = 1

            prev_date = current_date

        discontinuity_map[last_discontinuity] = continuous_count
        return discontinuity_map

    # Load CSV with datetime values
    df = pd.read_csv(f"/content/test/{file_name}.csv")  # Replace with your CSV file
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Find the continuous date ranges
    discontinuity_map = find_continuous_date_ranges(df)
    print("Discontinuity Map:", discontinuity_map)

    # Function to make predictions for each continuous range
    def predict_for_range(model, test_data, time_step, pred_days=30):
        x_input = test_data[len(test_data)-time_step:].reshape(1, -1)
        temp_input = list(x_input)
        temp_input = temp_input[0].tolist()

        lst_output = []
        n_steps = time_step
        i = 0

        while i < pred_days:
            if len(temp_input) > time_step:
                x_input = np.array(temp_input[1:])
                x_input = x_input.reshape(1, -1)
                x_input = x_input.reshape((1, n_steps, 1))
                yhat = model.predict(x_input, verbose=0)
                temp_input.extend(yhat[0].tolist())
                temp_input = temp_input[1:]
                lst_output.extend(yhat.tolist())
                i += 1
            else:
                x_input = np.array(temp_input).reshape((1, n_steps, 1))
                yhat = model.predict(x_input, verbose=0)
                temp_input.extend(yhat[0].tolist())
                lst_output.extend(yhat.tolist())
                i += 1

        return lst_output

    # Initialize list to store the results for all date ranges
    results = []

    # Loop over discontinuity map to predict for all ranges
    for start_date, time_step in discontinuity_map.items():
        # Assuming 'test_data' is your input data associated with this date range
        print(f"Predicting for date range starting at {start_date} with time_step {time_step}")

        predictions = predict_for_range(model, test_data, time_step, pred_days=30)

        # For each prediction, calculate the corresponding date
        for day_offset, prediction in enumerate(predictions):
            prediction_date = start_date + timedelta(days=day_offset)
            results.append({'date': prediction_date, 'prediction': prediction[0]})  # Assuming prediction is a list

    # Convert results to DataFrame
    predictions_df = pd.DataFrame(results)

    # Save the DataFrame to a CSV file
    predictions_df.to_csv(f"predicted_dates_{file_name}.csv", index=False)

    print("Predictions saved to 'predicted_dates.csv'.")

Epoch 1/100


<ipython-input-15-4340925e8800>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sd=maindf.iloc[0][0]
<ipython-input-15-4340925e8800>:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ed=maindf.iloc[-1][0]
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 233ms/step - loss: 0.0461 - val_loss: 0.1455
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0340 - val_loss: 0.1327
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0383 - val_loss: 0.1200
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0287 - val_loss: 0.1077
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0251 - val_loss: 0.0956
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0183 - val_loss: 0.0840
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0234 - val_loss: 0.0727
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0146 - val_loss: 0.0627
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0129 - val_loss: 0.0537
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0145 - val_loss: 0.0461
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0151 - val_loss: 0.0398
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0087 - val_loss: 0.0357
